In [12]:
import sys
from constants import PATH_INFO
sys.path.append(PATH_INFO.get('project_dir'))

# Optuna

- sampler： BO アルゴリズム

```
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)
```

- objective: 目的変数の実現値を受け取って，評価値を返す
  
```
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2
```

In [7]:
import optuna

# 目的関数の定義
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

# TPE Samplerの指定
sampler = optuna.samplers.TPESampler()

# ログレベルをINFOに設定して詳細な出力を有効にする
# optuna.logging.set_verbosity(optuna.logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.INFO)

# スタディの作成
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=5)

# 結果の表示
print(f'Best value: {study.best_value}')
print(f'Best params: {study.best_params}')
print()
print()

# 可視化
optuna.visualization.plot_optimization_history(study).show()

[I 2024-07-20 16:51:12,260] A new study created in memory with name: no-name-50f80134-9dce-4c2c-9ec2-851900ed49a0
[I 2024-07-20 16:51:12,267] Trial 0 finished with value: 62.37094533953334 and parameters: {'x': -5.897527799225106}. Best is trial 0 with value: 62.37094533953334.
[I 2024-07-20 16:51:12,269] Trial 1 finished with value: 54.951662333902156 and parameters: {'x': 9.41293884595726}. Best is trial 1 with value: 54.951662333902156.
[I 2024-07-20 16:51:12,271] Trial 2 finished with value: 84.9894103776851 and parameters: {'x': -7.2189701365003405}. Best is trial 1 with value: 54.951662333902156.
[I 2024-07-20 16:51:12,273] Trial 3 finished with value: 38.05571614029553 and parameters: {'x': -4.168931523391675}. Best is trial 3 with value: 38.05571614029553.
[I 2024-07-20 16:51:12,275] Trial 4 finished with value: 56.81248654372552 and parameters: {'x': 9.537405823207711}. Best is trial 3 with value: 38.05571614029553.


Best value: 38.05571614029553
Best params: {'x': -4.168931523391675}




In [14]:
from src.utils_warcraft import navigate_through_matrix
from src.utils_warcraft import manhattan_distance


def objective(trial):
    map_shape = (3,3)

    weights = np.array(
        [
            [0.1, 0.4, 0.9],
            [0.4, 0.1, 0.4],
            [0.9, 0.4, 0.1]
        ]
    )

    def _sample_objective_variables(map_shape=map_shape):
        directions = ['oo', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd'] # search space
        direction_matrix = np.zeros(map_shape, dtype=object)
        for i in range(map_shape[0]):
            for j in range(map_shape[1]):
                direction_matrix[i, j] = trial.suggest_categorical(f'({i},{j})', directions)
        return direction_matrix

    direction_matrix = _sample_objective_variables()
    print(f'direction matrix:\n{direction_matrix}\n\n')

    start = (0,0)
    goal = (map_shape[0]-1, map_shape[1]-1)
    history = navigate_through_matrix(direction_matrix, start, goal)

    if history:
        path_weight = sum(weights[coord] for coord in history)
        norm_const = manhattan_distance(start, goal)
        score1 = 1 - manhattan_distance(history[-1], goal) / norm_const + path_weight
    else:
        score1 = 1

    # Compute the score
    score2 = 0.0
    for i in range(3):
        for j in range(3):
            direction = direction_matrix[i, j]
            if direction == 'oo':
                continue
            score2 += weights[i, j]
    
    return score1 + score2

In [15]:
import numpy as np
from src.utils_warcraft import navigate_through_matrix, manhattan_distance


class BaseObjective:
    def __init__(self):
        raise NotImplementedError("Subclasses should implement this method")

    def sample(self, trial):
        raise NotImplementedError("Subclasses should implement this method")

    def evaluate(self):
        raise NotImplementedError("Subclasses should implement this method")

    def __call__(self, trial):
        raise NotImplementedError("Subclasses should implement this method")
    

class WarcraftObjective(BaseObjective):
    def __init__(self, weight_matrix: np.ndarray):
        self.map_shape = weight_matrix.shape
        self.weights = weight_matrix

    def sample(self, trial):
        directions = ['oo', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd']  # search space
        direction_matrix = np.zeros(self.map_shape, dtype=object)
        for i in range(self.map_shape[0]):
            for j in range(self.map_shape[1]):
                direction_matrix[i, j] = trial.suggest_categorical(f'({i},{j})', directions)
        self.direction_matrix = direction_matrix

    def evaluate(self):
        start = (0, 0)
        goal = (self.map_shape[0] - 1, self.map_shape[1] - 1)
        history = navigate_through_matrix(self.direction_matrix, start, goal)

        if history:
            path_weight = sum(self.weights[coord] for coord in history)
            norm_const = manhattan_distance(start, goal)
            loss1 = 1 - manhattan_distance(history[-1], goal) / norm_const + path_weight
        else:
            loss1 = 1

        mask = self.direction_matrix != 'oo'
        loss2 = self.weights[mask].sum()
        
        return loss1 + loss2

    def __call__(self, trial):
        self.sample(trial)
        print(f'direction matrix:\n{self.direction_matrix}\n\n')
        return self.evaluate()

In [ ]:
import numpy as np
from src.utils_warcraft import navigate_through_matrix, manhattan_distance

class BaseObjective:
    def __init__(self):
        raise NotImplementedError("Subclasses should implement this method")

    def sample(self, trial):
        raise NotImplementedError("Subclasses should implement this method")

    def evaluate(self):
        raise NotImplementedError("Subclasses should implement this method")

    def __call__(self, trial):
        raise NotImplementedError("Subclasses should implement this method")

class WarcraftObjective(BaseObjective):
    def __init__(self, weight_matrix: np.ndarray):
        self.map_shape = weight_matrix.shape
        self.weights = weight_matrix

    def sample(self, trial):
        directions = ['oo', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd']  # search space
        direction_matrix = np.zeros(self.map_shape, dtype=object)
        for i in range(self.map_shape[0]):
            for j in range(self.map_shape[1]):
                direction_matrix[i, j] = trial.suggest_categorical(f'({i},{j})', directions)
        self.direction_matrix = direction_matrix

    def evaluate(self):
        start = (0, 0)
        goal = (self.map_shape[0] - 1, self.map_shape[1] - 1)
        history = navigate_through_matrix(self.direction_matrix, start, goal)

        if history:
            path_weight = sum(self.weights[coord] for coord in history)
            norm_const = manhattan_distance(start, goal)
            loss1 = 1 - manhattan_distance(history[-1], goal) / norm_const + path_weight
        else:
            loss1 = 1

        mask = self.direction_matrix != 'oo'
        loss2 = self.weights[mask].sum()
        
        return loss1 + loss2

    def __call__(self, trial):
        self.sample(trial)
        print(f'direction matrix:\n{self.direction_matrix}\n\n')
        return self.evaluate()

In [ ]:
# Import required libs.
import numpy as np
import optuna
import pickle
import plotly.graph_objects as go
import itertools


def objective(trial):
    """目的関数を定義する。ここで、パラメータx, yを探索する範囲を指定する。ここでは、例として、`-5 < x, y < 5と`を指定。"""
    x = trial.suggest_float(name="x", low=-5.0, high=5.0, step=0.1)
    y = trial.suggest_float(name="y", low=-5.0, high=5.0, step=0.1)
    return _sphereFunction(x=x, y=y)

def _sphereFunction(x, y):
    """f(x, y) = x^2 + y^2の値を返す関数を定義する。目的関数を、簡単に、変えられるように、このような形にしている。"""
    return pow(x, 2) + pow(y, 2)


# Create study for function optimization.
study = optuna.create_study()

# Run additional 10 trials. We can continue optimization just by calling optimize() using same study object. 
n_trials = 10
study.optimize(objective, n_trials=n_trials)

# Run additional trials in 1 seconds. We can continue optimization just by calling optimize() using same study object. 
timeout = 1.0
study.optimize(objective, timeout=timeout)

# Show optimization result. The results of previous trials is also included in the figure.
print(f"  Optimized value : {study.best_value}")
print(f"  Parameters : {study.best_params}")

def toPickle(obj, path_to_pickle):
    """objをPickleファイルとして保存する。"""
    with open(path_to_pickle, "wb") as fout:
        pickle.dump(obj, fout)

# Save optimization history (study object) as pickle file.
path_to_study = "/kaggle/working/study.pkl"
toPickle(obj=study, path_to_pickle=path_to_study)
!ls {path_to_study}

def fromPickle(path_to_pickle):
    """Load obj from pickle file."""
    with open(path_to_pickle, "rb") as fin:
        return pickle.load(fin)

# Reload saved study object and restart optimization (Run additional 1000 trials. It is
# meaningless calculation, just an example for showing how to restart optimization.).
study_from_pickle = fromPickle(path_to_pickle=path_to_study)
n_trials = 1000
study_from_pickle.optimize(objective, n_trials=n_trials)

